<img src="Tarjeta.png">

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Generamos-un-ARMA-y-lo-modelamos" data-toc-modified-id="Generamos-un-ARMA-y-lo-modelamos-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Generamos un ARMA y lo modelamos</a></span><ul class="toc-item"><li><span><a href="#Analisis-de-Estacionariedad" data-toc-modified-id="Analisis-de-Estacionariedad-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Analisis de Estacionariedad</a></span></li><li><span><a href="#Autocorrelación.y-Autocorrelación-Parcial" data-toc-modified-id="Autocorrelación.y-Autocorrelación-Parcial-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Autocorrelación.y Autocorrelación Parcial</a></span></li><li><span><a href="#Fittear-el-modelo" data-toc-modified-id="Fittear-el-modelo-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Fittear el modelo</a></span></li></ul></li><li><span><a href="#Diagnostico" data-toc-modified-id="Diagnostico-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Diagnostico</a></span></li><li><span><a href="#Predicción" data-toc-modified-id="Predicción-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Predicción</a></span></li><li><span><a href="#Autocorrelacion" data-toc-modified-id="Autocorrelacion-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Autocorrelacion</a></span></li><li><span><a href="#Modelos-ARIMA" data-toc-modified-id="Modelos-ARIMA-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Modelos ARIMA</a></span></li><li><span><a href="#Modelos-ARIMAx" data-toc-modified-id="Modelos-ARIMAx-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Modelos ARIMAx</a></span></li><li><span><a href="#Procedimiento-Completo" data-toc-modified-id="Procedimiento-Completo-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Procedimiento Completo</a></span><ul class="toc-item"><li><span><a href="#Lectura-de-datos" data-toc-modified-id="Lectura-de-datos-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Lectura de datos</a></span></li></ul></li><li><span><a href="#Test-Serie-Estacionaria" data-toc-modified-id="Test-Serie-Estacionaria-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Test Serie Estacionaria</a></span></li><li><span><a href="#Autocorrelacion" data-toc-modified-id="Autocorrelacion-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Autocorrelacion</a></span></li></ul></div>

# Intro

In [ ]:
import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)

Los modelos ARIMA son la forma más tradicional de resolver los modelos temporales. Iremos uno por uno revisando los modelos que lo componen, junto con un ejemplo final. ya sea para hacer previsiones o para comprender mejor los datos. No cubriremos toda la teoría detrás del modelo ARIMA, pero le mostraremos cuáles son los pasos que debe seguir para aplicarlo correctamente.AR: Autoregresión. Indica que la serie temporal se hace una regresión sobre sus propios valores retardados.
I: Integrada. Indica que los valores de los datos se han sustituido por la diferencia entre sus valores y los anteriores para convertir la serie en estacionaria.
MA: Media móvil. Indica que el error de regresión es en realidad una combinación lineal de términos de error cuyos valores se produjeron contemporáneamente y en varios momentos del pasado.








In [1]:
import pandas as pd 
import numpy as np
import plotly.express as px
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.tsa.arima.model import ARIMA

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from matplotlib.pylab import rcParams
from pmdarima import auto_arima
from statsmodels.tsa.arima_model import ARIMA,ARIMAResults,ARMA,ARMAResults
#import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX

Habrá que repetir algunos de los modelos con el siguiente dataset

In [ ]:
link_pax = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv'
df_pax =  pd.read_csv(link_pax, parse_dates = ['Month'], index_col = 'Month')
df_pax.head(5)

# Modelos Autorregreisvos (AR)

Modelos de Autoregresion

## Generamos un ARMA y lo modelamos

Tanto el componente AR como el MA deben incluir el coeficiente del lag cero. En casi todos los casos estos valores deben ser 1. Además, debido a la utilización de la representación lag-polinomio, los parámetros AR deben tener el signo contrario al que se escribiría en la representación ARMA. Vea los ejemplos siguientes.

El proceso ARMA(p,q) se describe mediante

𝑦𝑡=𝜙1𝑦𝑡−1+…+𝜙𝑝𝑦𝑡−𝑝+𝜃1𝜖𝑡−1+…+𝜃𝑞𝜖𝑡−𝑞+𝜖𝑡 <br>
y la parametrización utilizada en esta función utiliza la representación lag-polinomio <br>
(1−𝜙1𝐿−…−𝜙𝑝𝐿𝑝)𝑦𝑡=(1+𝜃1𝐿+…+𝜃𝑞𝐿𝑞)𝜖𝑡

In [ ]:
from statsmodels.tsa.arima_process import ArmaProcess
ar1 = np.array([1, -0.8])
ma1 = np.array([1])
AR_object1 = ArmaProcess(ar1, ma1)
serie_ar1 = AR_object1.generate_sample(nsample=1000)

In [ ]:
px.line(serie_ar1)

### Analisis de Estacionariedad

In [ ]:
from statsmodels.tsa.stattools import adfuller
result=adfuller(serie_ar1)
#to help you, we added the names of every value
dict(zip(['adf', 'pvalue', 'usedlag', 'nobs', 'critical' 'values', 'icbest'],result))

### Autocorrelación.y Autocorrelación Parcial

In [ ]:
plot_acf(serie_ar1, alpha=1, lags=20)
plt.show
plot_pacf(serie_ar1, alpha=1, lags=20)
plt.show

### Fittear el modelo 

In [ ]:
# Fit an AR(1) model to the first simulated data
mod = ARIMA(serie_ar1, order=(1,0,0))

In [ ]:
res = mod.fit()

In [ ]:
print(res.summary())

## Diagnostico

Hacer el diagnostico de los resultados

In [ ]:
res.plot_diagnostics()
plt.show()

¿Cómo interpretar el resultado del plot_diagnostics?

+ **Arriba a la izquierda:** Los errores residuales parecen fluctuar en torno a una media de cero y tienen una varianza uniforme.
+ **Arriba a la derecha:** El gráfico de densidad sugiere una distribución normal con media cero.
+ **Abajo a la izquierda:** Todos los puntos deberían estar perfectamente alineados con la línea roja. Cualquier desviación significativa implicaría que la distribución está sesgada.
+ **Abajo a la derecha:** El correlograma, también conocido como gráfico ACF, muestra que los errores residuales no están autocorrelacionados. Cualquier autocorrelación implicaría que hay algún patrón en los errores residuales que no se explica en el modelo. Así que tendrá que buscar más X (predictores) para el modelo.



## Predicción

In [ ]:
fitted = res.predict(n_periods=12, return_conf_int=True)
a = res.forecast(steps = 100)
a

In [ ]:
print("Qué parámetros trenemos?")
print(res.params)

<hr style="height:5px;border-width:0;color:orange;background-color:orange">

# Modelos de Medias Moviles (MA)

In [ ]:
#from statsmodels.tsa.arima_process import ArmaProcess

# Plot 1: MA parameter = -0.9

ar1 = np.array([1])
ma1 = np.array([1, -0.9])
MA_object1 = ArmaProcess(ar1, ma1)
serie_ma1= MA_object1.generate_sample(nsample=1000)
px.line(serie_ma1)



In [ ]:
# Plot 2: MA parameter = +0.9

ar2 = np.array([1])
ma2 = np.array([1, 0.9])
MA_object2 = ArmaProcess(ar2, ma2)
serie_ma2 = MA_object2.generate_sample(nsample=1000)
px.line(serie_ma2)

## Autocorrelacion

Primero empezamos con -0.9

In [ ]:
plot_acf(serie_ma1, lags=20)
plt.show()

In [ ]:
plot_pacf(serie_ma1, lags=20)
plt.show()

***

In [ ]:
plot_acf(serie_ma2, lags=20)
plt.show()

In [ ]:
plot_acf(serie_ma2, lags=20)
plt.show()

***

In [ ]:

mod = ARIMA(serie_ma1, order=(0,0,1))
res = mod.fit()

# Print out summary information on the fit
print(res.summary())




In [ ]:
mod = ARIMA(serie_ma2, order=(0,0,1))
res = mod.fit()

# Print out summary information on the fit
print(res.summary())



In [ ]:

res.plot_diagnostics()
plt.show()

## Modelos ARIMA

Para los datos no estacionales los parámetros son
+ p: El número de observaciones de retraso que utilizará el modelo
+ d: El número de veces que se diferencian las observaciones brutas hasta la estacionariedad.
+ q: El tamaño de la ventana de la media móvil.

<hr style="height:5px;border-width:0;color:orange;background-color:orange">

## Modelos ARIMAx

In [ ]:
# Instantiate the model
model = SARIMAX(hospital['wait_times_hrs'], order = [2,0,1], exog= hospital['nurse_count']  )

# Fit the model
results = model.fit()

# Print model fit summary
print(results.summary(
))

In [ ]:
hospital = pd.DataFrame([1.75, 1.66, 1.65, 1.62, 1.48, 1.77, 1.99, 
                         2.17, 1.57, 1.28, 1.44, 1.06, 1.06, 1.2, 
                         0.89, 1.04, 0.77, 0.54, 1.23, 1.33, 1.65, 
                         1.27, 1.26, 1.4, 1.51, 2.13, 2.35, 2.53, 
                         2.19, 1.72, 1.55, 1.19, 0.96, 1.1, 1.16,
                         1.03, 0.71, 0.82, 1.0, 1.51, 1.25, 1.07,
                         0.69, 1.26, 1.73, 1.76, 1.6, 1.59, 2.32,
                         2.41, 1.95, 1.06, 1.24, 1.61, 1.53, 1.26,
                         0.72, 0.71, 0.59, 0.26, 0.61, 0.66, 0.61, 
                         0.97, 1.2, 1.26, 1.0, 0.58, 1.17, 1.81, 2.13, 
                         1.19, 1.38, 1.54, 1.75, 1.74, 1.39, 0.87, 1.66,
                         1.72, 1.48, 1.73, 1.45, 1.0, 1.23, 1.4, 1.05, 
                         0.67, 0.5, 1.13, 1.74, 2.69, 2.29, 2.28, 2.52, 
                         1.92, 1.91, 1.66, 1.98, 1.9, 1.4, 1.01, 1.21, 1.46, 
                         1.8, 1.3, 1.02, 1.46, 1.6, 1.63, 1.47, 1.37, 1.22,
                         1.38, 1.6, 2.44, 2.45, 2.02, 1.72, 1.49, 1.4,
                         1.32, 1.69, 2.01, 2.24, 1.86, 1.4, 1.67, 2.14, 
                         1.51, 1.09, 1.24, 1.66, 1.28, 0.99, 1.15, 1.28, 
                         0.96, 1.3, 1.28, 1.71, 1.56, 1.17, 1.36, 1.78, 2.08,
                         1.97, 2.0, 1.97, 2.02, 1.59, 1.21, 0.86, 0.19, 0.76, 1.08, 0.8, 0.57, 0.94,
                         1.37, 1.61, 1.96, 1.56, 1.1, 1.6, 1.71, 1.29, 1.55], 
                        columns = ['wait_times_hrs'])
hospital['nurse_count'] = [1.0, 1.0, 1.0, 1.0, 1.0, 3.0, 3.0, 3.0, 7.0, 9.0, 9.0,
                           9.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0,
                           9.0, 9.0, 7.0, 7.0, 5.0, 5.0, 3.0, 3.0, 3.0, 5.0, 
                           5.0, 5.0, 7.0, 7.0, 7.0, 7.0, 7.0, 9.0, 9.0, 9.0, 
                           9.0, 9.0, 9.0, 5.0, 5.0, 5.0, 5.0, 5.0, 1.0, 1.0, 
                           1.0, 3.0, 3.0, 5.0, 5.0, 5.0, 9.0, 11.0, 11.0, 
                           11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 
                           11.0, 9.0, 9.0, 7.0, 7.0, 5.0, 5.0, 5.0, 5.0, 5.0,
                           7.0, 5.0, 5.0, 7.0, 7.0, 7.0, 7.0, 5.0, 7.0, 7.0, 
                           7.0, 7.0, 7.0, 7.0, 3.0, 3.0, 3.0, 3.0, 5.0, 3.0, 3.0,
                           3.0, 3.0, 3.0, 5.0, 5.0, 5.0, 7.0, 7.0, 7.0, 7.0, 7.0,
                           7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 3.0, 3.0, 3.0, 3.0, 3.0, 
                           1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 5.0, 5.0, 
                           5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 3.0, 3.0, 
                           5.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 
                           7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 
                           5.0, 5.0, 5.0, 3.0]## Modelos Sarima

In [ ]:
model = SARIMAX(hospital['wait_times_hrs'], order = [2,0,1], exog
 = hospital['nurse_count']  )

# Fit the model
results = model.fit()

# Print model fit summary
print(results.summary(
))

In [ ]:
a = results.forecast(steps = 4, exog = [4,5,2,3], alpha= 0.95)


In [ ]:

pd.DataFrame(a)

In [ ]:
results.plot_diagnostics()
plt.show()

<hr style="height:5px;border-width:0;color:orange;background-color:orange">

# Ejemplo  

## Procedimiento Completo

### Lectura de datos

In [ ]:
link_pax = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv'
df_pax =  pd.read_csv(link_pax, parse_dates = ['Month'], index_col = 'Month')
df_pax.head(5)

In [ ]:
px.line(df_pax['Passengers'])

## Test Serie Estacionaria

Como puede ver claramente, hay una tendencia en el tiempo y eso sugiere que los datos no son estacionarios. Sin embargo, para estar seguros, utilizaremos una prueba de Dickey-Fuller aumentada.

In [ ]:
from statsmodels.tsa.stattools import adfuller
result=adfuller(df_pax['Passengers'])
#to help you, we added the names of every value
dict(zip(['adf', 'pvalue', 'usedlag', 'nobs', 'critical' 'values', 'icbest'],result))


Como ya habíamos visto un p-valor muy alto 1. Por lo que podemos decir que no es estacionario. The next step is to transform our data to Stationary so we will have an estimate for d and D parameters we will use in the model. This can be done using Differencing and it’s performed by subtracting the previous observation from the current observation.

+ difference(T) = observation(T) — observation(T-1)

In [ ]:
df_pax['diff1']=df_pax['Passengers']-df_pax['Passengers'].shift(1)
df_pax['diff1'].plot()

A continuación, comprobaremos de nuevo la estacionariedad mediante la prueba de Dickey-Fuller aumentada y, si es estacionaria, pasaremos al siguiente paso. Si no lo es, aplicaremos la diferenciación de nuevo hasta que tengamos una serie estacionaria. La diferenciación se puede hacer muy fácilmente con pandas utilizando la función shift.

Parece que hemos eliminado la tendencia y la serie es estacionaria. Sin embargo, utilizaremos la prueba de Dickey-Fuller aumentada para demostrarlo.

In [ ]:
result=adfuller(df_pax['diff1'].dropna())
dict(zip(['adf', 'pvalue', 'usedlag', 'nobs', 'critical' 'values', 'icbest'],result))


Como puede ver, no rechazamos la hipótesis nula porque tenemos un valor p>0,05. Esto sugiere que la serie no es estacionaria y que tenemos que volver a utilizar la diferenciación tomando la segunda diferencia. La segunda diferencia se puede calcular como la primera, pero esta vez, en lugar de utilizar las observaciones, utilizaremos la primera diferencia.
Obtengamos los resultados de la prueba Dickey-Fuller aumentada para la segunda diferencia.

In [ ]:
df_pax['diff2']=df_pax['diff1']-df_pax['diff1'].shift(1)
df_pax['diff2'].plot()

In [ ]:
result=adfuller(df_pax['diff2'].dropna())
dict(zip(['adf', 'pvalue', 'usedlag', 'nobs', 'critical' 'values', 'icbest'],result))


Haremos lo mismo con la estacionalidad

In [ ]:
df_pax['Seasonal_Difference']=df_pax['Passengers']-df_pax['Passengers'].shift(12)
ax=df_pax['Seasonal_Difference'].plot()

El valor p es inferior a 0,05, por lo que podemos rechazar la hipótesis nula. Esto significa que la segunda diferencia es estacionaria y sugiere que una buena estimación del valor d es 2.
Nuestros datos son estacionales, por lo que necesitamos estimar también el valor D, que es el mismo que el valor d pero para la diferencia estacional. La diferencia estacional puede calcularse desplazando los datos por el número de filas por temporada (en nuestro ejemplo 12 meses por año) y restándolos de la temporada anterior. Esta no es la primera diferencia estacional. Si conseguimos que la diferencia estacional sea estacionaria entonces el valor D será 0. Si no es así entonces calcularemos la primera diferencia estacional.
diferencia estacional(T) = observación(T) - observación(T-12)
primera diferencia estacional(T) = diferencia estacional(T) - diferencia estacional(T-1)

Traducción realizada con la versión gratuita del traductor www.DeepL.com/Translator

In [ ]:
result=adfuller((df_pax['Seasonal_Difference']).dropna())
dict(zip(['adf', 'pvalue', 'usedlag', 'nobs', 'critical' 'values', 'icbest'],result))


The p-value is less than 0.05 thus it’s stationary and we don’t have to use differencing. That suggests using 0 for the D value.

## Autocorrelacion

El último paso antes del modelo ARIMA es crear los Gráficos de Autocorrelación y Autocorrelación Parcial para ayudarnos a estimar los parámetros p,q, P y Q.
Hay algunas reglas muy útiles para los modelos ARIMA y ARIMA Estacional que estamos utilizando para ayudarnos a estimar los parámetros mirando los Gráficos de Autocorrelación y Autocorrelación Parcial. Crearemos los gráficos para la segunda diferencia y la diferencia estacional de nuestras series de tiempo porque estas son las series estacionarias que terminamos usando en ARIMA (d=2, D=0).

In [ ]:

fig1=plot_acf(df_pax['diff2'].dropna())
fig2=plot_pacf(df_pax['diff2'].dropna())

Podemos ver que tenemos un corte brusco en el lag-1 en ambos gráficos. De acuerdo con las reglas que mencionamos anteriormente, esto sugiere el uso de un término AR y MA. En otras palabras, p=1 y q=1.
Ahora, necesitamos los mismos gráficos de diferencia estacional.

In [ ]:
fig1=plot_acf(df_pax['Seasonal_Difference'].dropna())
fig2=plot_pacf(df_pax['Seasonal_Difference'].dropna())

Tenemos una disminución gradual en el gráfico de autocorrelación y un corte en el gráfico de autocorrelación parcial. Esto sugiere utilizar AR y no sobrepasar el valor de 1 para la parte estacional del ARIMA.
Los valores que hemos elegido pueden no ser óptimos. Puede jugar con estos parámetros para afinar el modelo teniendo como guía las reglas que mencionamos anteriormente.


In [ ]:

model=SARIMAX(df_pax['Passengers'],order=(1,2,1),seasonal_order=(1, 0, 0, 12))
result=model.fit()

In [ ]:
result.resid.plot(kind='kde')

In [ ]:
## Prediccion

In [ ]:
from pandas.tseries.offsets import DateOffset
new_dates=[df_pax.index[-1]+DateOffset(months=x) for x in range(1,48)]
df_pred=pd.DataFrame(index=new_dates,columns =df_pax.columns)
df_pred.head()

In [ ]:
df2=pd.concat([df_pax,df_pred])
df2['predictions']=result.predict(start=0,end=191)
df2[['Passengers','predictions']].plot()

In [ ]:
from sklearn.metrics import mean_absolute_error
df2 = df2[~(df2['diff2'].isna()) & ~(df2['predictions'].isna())]
mean_absolute_error(df2['Passengers'], df2['predictions'])

In [ ]:
result=adfuller((df_pax['diff2']).dropna())
dict(zip(['adf', 'pvalue', 'usedlag', 'nobs', 'critical' 'values', 'icbest'],result))


<hr style="height:5px;border-width:0;color:orange;background-color:orange">

In [ ]:
link = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv'
df = pd.read_csv(link)

In [ ]:
df['passengers'] = df['Passengers'].diff()
df = df[~df['passengers'].isna()]

In [ ]:

Sarimax_model = auto_arima(df_pax['Passengers'],
                       start_P=1,
                       start_q=1,
                       max_p=3,
                       max_q=3,
                       m=12,
                       seasonal=True,
                       D=1,
                       max_d = 3,
                       trace=True,
                       error_action='ignore',
                       suppress_warnings=True,
                       stepwise=True)
Sarimax_model.summary()

In [ ]:
model = SARIMAX(df_pax['Passengers'],order=(2, 1, 1),
              seasonal_order=(0, 1, 0, 12),
              enforce_stationarity=False,
              enforce_invertibility=False)
results = model.fit()

In [ ]:
from pandas.tseries.offsets import DateOffset
new_dates=[df_pax.index[-1]+DateOffset(months=x) for x in range(1,48)]
df_pred=pd.DataFrame(index=new_dates,columns =df_pax.columns)
df_pred.head()

In [ ]:
type(Sarimax_model)

In [ ]:
forecast = results.predict(start = len(df),
                           end=len(df)+24,
                           typ='levels').rename('data sarimax (1,0,1) forecast')

In [ ]:
df_pax['Passengers'].plot(figsize=(12,8),legend=True)
forecast.plot(legend=True)
plt.show()
results.plot_diagnostics()
plt.show()

In [ ]:
results.resid.plot(kind='kde')

In [ ]:
from pandas.tseries.offsets import DateOffset
new_dates=[df_pax.index[-1]+DateOffset(months=x) for x in range(1,48)]
df_pred=pd.DataFrame(index=new_dates,columns =df_pax.columns)
df_pred.head()

In [ ]:
df2=pd.concat([df_pax,df_pred])
df2['predictions']=results.predict(start=0,end=191)
df2[['Passengers','predictions']].plot()

Calculamos el Erro

In [ ]:
from sklearn.metrics import mean_absolute_error
df2 = df2[~(df2['diff2'].isna()) & ~(df2['predictions'].isna())]
mean_absolute_error(df2['Passengers'], df2['predictions'])